In [1]:
#vamos a clasificar imagenes de números escritos a mano
import torch
import torch.nn as nn
import torchvision.datasets as datasets #para bajar los datos
import torchvision.transforms as transforms
from torch.autograd import Variable

c:\Users\52333\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] No se encontró el proceso especificado
  warn(f"Failed to load image Python extension: {e}")


In [2]:
input_size = 784 #input neurons (es igual al número de pixeles en las imágenes)
hidden_size=50 #es input+output/2, es una buena forma de aproximar las hidden neurons
out_size=10 
epochs=10
batch_size=100  #input size in one iteration
learning_rate=0.001

In [3]:
#bajamos nuestro set de datos
train_dataset=datasets.MNIST(root='./data', #folder donde guardaremos las imagenes
                            train=True, #porque queremos bajar el training set
                            transform=transforms.ToTensor(), #originalmente son matrices, entonces las pasamos a tensores
                            download=True) 

#y ahora bajamos el set para las pruebas
test_dataset=datasets.MNIST(root='./data', #folder donde guardaremos las imagenes
                            train=False, #porque queremos bajar el training set
                            transform=transforms.ToTensor(), #originalmente son matrices, entonces las pasamos a tensores
                            download=True) 


In [4]:
#para que los datos sean iterables, se carga loader. Se hace un shuffle a los datos para que sean independientes del orden
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                        batch_size=batch_size,
                                        shuffle=True)

test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=batch_size,
                                        shuffle=False) #como solo estqamos ya probando, no importa ya lo del orden


In [28]:
class MyBackwards(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input):
        output=torch.sigmoid(input)
        ctx.save_for_backward(input)
        return output
    
    def backward(ctx,grad_output):
        input,=ctx.saved_tensors
        #grad_input=grad_output.clone()
        return grad_output*(torch.sigmoid(input))*(1-torch.sigmoid(input))


In [29]:
#Ahora construimos nuestra red neuronal
class Net(nn.Module): #hacemos nuestra propia clase Net pero la heredamos de la clase nn.Module
    def __init__(self,input_size,hidden_size,out_size):
        super(Net,self).__init__() #usamos super para heredar nn.module, hay que pasarle la función Net y los objetos self
        #ahora vamos a definir todas las capas
        self.fc1=nn.Linear(input_size,hidden_size) #la primera capa, es fully connected
        self.activation=MyBackwards.apply # activación para todas las capas. Este atributo lo podemos llamar las veces que queramos en forward
        self.fc2=nn.Linear(hidden_size,out_size) #segunda capa, fully connected
        
       
    
    #definimos un método de la clase
    def forward(self,x):
        out=self.fc1(x) #propagamos los inputs en nuestra primer capa
        out=self.activation(out) #los activamos
        out=self.fc2(out)  #los pasamos por la hidden layer
        return out
        

In [30]:
#Ahora creamos un objeto de esa clase, el cual será nuestra red
net=Net(input_size,hidden_size,out_size)
CUDA=torch.cuda.is_available()
if CUDA:
    net=net.cuda()

#ponemos la funcion de coste. La cross entropy viene con softmax, entonces ya no se necesita especificar softmax
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(net.parameters(),lr=learning_rate)

    
    

In [31]:
#veamos cómo está construida nuestra red
net.parameters

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)>

In [32]:
#visualizamos el train loader

#si vemos la carpeta donde bajamos los datos, vemos que images y labels son los nombres de los archivos
for i,(images,labels) in enumerate(train_loader):
    print(images.size())
    #Returns a new tensor with the same data as the self tensor but of a different shape.
    #The returned tensor shares the same data and must have the same number of elements, but may have a different size. 
    images=images.view(-1,784) #el -1 es para que se infiera esa dimension a partir de las otras
    print(images.size())
    
    #nos va a salir [100,1,28,28], 100 imagenes, 1 canal (porque es blanco y negro) y 28*28 pixeles
    #en images.views nos da 100 imaagenes con 784 enetradas
    

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([

In [33]:
#Ahora si vamos a entrenar la red
#por ejemplo en keras o tensor flow esto se corre en el background
correct_train=0 #cuantas clasificaciones correctas tenemos, al principio esto es cero, en cada iterción crecerá
total_train=0 
for epoch in range(epochs):
    for i,(images,labels) in enumerate(train_loader):
        #vamos a aplanar la imagen: (batch,1,28,28 )--> (100,784) y lo envolvemos en Variable(...)
        images=Variable(images.view(-1,28*28))
        labels=Variable(labels)
        if CUDA:
            images=images.cuda()
            labels=labels.cuda()
        #limpiamos los gradientes para que no se acumulen: param=param-lr*param_grad
        optimizer.zero_grad()
        
        #corremos las imagenes en la red en la función del forward, la cual nos daba de return un output
        outputs=net(images)
        
        #y tomamos el máximo de esos outputs
        _,predicted=torch.max(outputs.data,1) # queremos el segundo argumento de torch.max, por eso usamos -, 
                                                #ese segundo argumento es el indice del maximo
                                                #usamos outputs.data porque output es un tensor y con eso le sacamos los valores
                                                #el 1 es el axis en el que queremos el maximo, o sea en los renglones
        
        
        #ahora calculamos la funcion de coste
        loss=criterion(outputs,labels)
        loss.backward() #hacemos backpropagation
        optimizer.step() #actualizamos los pesos
        
        #imprimimos los resultados de cada epoca
        #para eso necesitamos lo siguiente:
        #ahora acumulamos nuestra variable de entrenamiento 
        total_train+=labels.size(0)
        
        #contamos las predicciones correctas
        if CUDA: #con cuda ocupamos pasar a cpu porque usamos .sum() y no hay implementacion de esto para gpu
            correct_train+=(predicted.cpu()==labels.cpu()).sum() #sumamos las veces los digitos de la prediccion son =  labels
        else:
            correct_train+=(predicted==labels).sum()
        if (i+1)%100==0:
            print('Epoca[{}/{}], Iteracion[{}/{}], Training Loss:{}, Training Accuracy: {}% '
                  .format(epoch+1,epochs,i+1,len(train_dataset)//batch_size,loss,(100*correct_train/total_train)))
print('Se terminó el entrenamiento')              

Epoca[1/10], Iteracion[100/600], Training Loss:1.3844194412231445, Training Accuracy: 58.36000061035156% 
Epoca[1/10], Iteracion[200/600], Training Loss:0.8285449147224426, Training Accuracy: 69.98500061035156% 
Epoca[1/10], Iteracion[300/600], Training Loss:0.7105832695960999, Training Accuracy: 75.33999633789062% 
Epoca[1/10], Iteracion[400/600], Training Loss:0.44498664140701294, Training Accuracy: 78.67500305175781% 
Epoca[1/10], Iteracion[500/600], Training Loss:0.3822510838508606, Training Accuracy: 80.83799743652344% 
Epoca[1/10], Iteracion[600/600], Training Loss:0.4184756577014923, Training Accuracy: 82.37166595458984% 
Epoca[2/10], Iteracion[100/600], Training Loss:0.37286365032196045, Training Accuracy: 83.62000274658203% 
Epoca[2/10], Iteracion[200/600], Training Loss:0.3253781795501709, Training Accuracy: 84.51875305175781% 
Epoca[2/10], Iteracion[300/600], Training Loss:0.4077731966972351, Training Accuracy: 85.31888580322266% 
Epoca[2/10], Iteracion[400/600], Training Lo

In [34]:
#Test the network (No necesitamos calcular la loss ni actualizar los pesos)
correct=0
total=0

for images, labels in test_loader: #iteramos en el test_loader
    images=Variable(images.view(-1,28*28)) #envolvemos el tensor y le damos un reshape
    if CUDA:
        images=images.cuda()
    #para cada input (sample/row) en el batch, el output contiene 10 elementos
    outputs=net(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    if CUDA: #con cuda ocupamos pasar a cpu porque usamos .sum() y no hay implementacion de esto para gpu
            correct+=(predicted.cpu()==labels.cpu()).sum() #sumamos las veces los digitos de la prediccion son =  labels
    else:
        correct+=(predicted==labels).sum()
print('Final Test Accuracy: %d%%'%(100*correct/total))

Final Test Accuracy: 95%
